In [1]:
import numpy as np
import opensim as osim
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import os

In [2]:
# Keeping pandas from truncating long strings
pd.set_option('display.max_colwidth', 10000000)

## Paths

In [4]:
os.getcwd()

'e:\\055\\P55\\model_update'

In [ ]:
# path to generic scaled model
scaled_model = f"../final_results/generic_scaled/scaled_model_joints.osim"

In [ ]:
# path to the new personalized model
new_model_name = f'tps_transformed.osim'
path_to_model = os.path.join("..", 'final_results', 'personalized')
if not os.path.exists(path_to_model):
    print(path_to_model)
    os.makedirs(path_to_model)
new_model = os.path.join(path_to_model, new_model_name)

In [ ]:
# IMPORTANT : THESE SHOULD BE PERSON-SPECIFIC
mass_text = '52.4'
height_text = '1.60'
age_text = '33'

# path to experimental .trc file : <marker_file>
experimental_markers = r'../motion_lab/static/Static03/task.trc'
static_df = pd.read_csv(experimental_markers, delimiter='\t', skiprows=3, header=[0,1], index_col=0)

In [ ]:
# path to TPS results
tps_folder = os.path.join("..", 'final_results')

'tps_warping_results\\results'

## Update Scaled Model with TPS data

### Import Transformed Markers

In [9]:
markers_df = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_l_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_r_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_bone_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_bone_markers.csv'), index_col=0)])
markers_df = markers_df.set_index('name')
markers_df.to_csv(os.path.join(tps_folder, 'markers_transformed.csv'))

### Import Transformed Muscle Paths

In [10]:
mscles_df = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_l_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'patella_r_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_muscle_paths.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_muscle_paths.csv'), index_col=0)])
mscles_df = mscles_df.set_index('name')
mscles_df.to_csv(os.path.join(tps_folder, 'muscles_transformed.csv'))

### Import Transformed Wrapping Surfaces

In [11]:
wrp_df  = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_wrap_translations.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_wrap_translations.csv'), index_col=0)])
wrp_df = wrp_df.set_index('name')
wrp_df.to_csv(os.path.join(tps_folder, 'wrp_transformed.csv'))

### Import transformed Skin Markers

In [12]:
skin_df = pd.concat([pd.read_csv(os.path.join(tps_folder, 'pelvis_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_l_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'femur_r_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_l_skin_markers.csv'), index_col=0),
            pd.read_csv(os.path.join(tps_folder, 'tibia_r_skin_markers.csv'), index_col=0)])
skin_df = skin_df.set_index('name')
skin_df.to_csv(os.path.join(tps_folder, 'skin_transformed.csv'))

### Parse current scaled model

In [13]:
tree=ET.parse(scaled_model)
root = tree.getroot()

### Update reference geometry

In [14]:
ref_geom_dic = {'body': [], 'Mesh name' : [], 'Mesh file' : []}

ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\r_pelvis.stl'))
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\l_pelvis.stl'))
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_3"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\sacrum.stl'))

ref_geom_dic['body'].append("femur_r"); ref_geom_dic['Mesh name'].append("femur_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\femur_r.stl'))
ref_geom_dic['body'].append("femur_l"); ref_geom_dic['Mesh name'].append("femur_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\femur_l.stl'))

ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\tibia_r.stl'))
ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\fibula_r.stl'))

ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\tibia_l.stl'))
ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_2"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\fibula_l.stl'))

ref_geom_dic['body'].append("patella_r"); ref_geom_dic['Mesh name'].append("patella_r_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\patella_r.stl'))
ref_geom_dic['body'].append("patella_l"); ref_geom_dic['Mesh name'].append("patella_l_geom_1"); ref_geom_dic['Mesh file'].append(os.path.join('bones\\patella_l.stl'))

In [15]:
ref_geom_df = pd.DataFrame(ref_geom_dic).set_index('Mesh name')

In [16]:
for Mesh in root.iter('Mesh'):
    point = Mesh.attrib['name']
    if point in ref_geom_df.index:
        new_text = ref_geom_df.loc[point, 'Mesh file']
        # print(point, new_text)
        Mesh.find('mesh_file').text = new_text
        Mesh.find('scale_factors').text = '1 1 1'
    else: pass

### Update muscle paths

In [17]:
muscle_path_names = []
muscle_path_old = []
muscle_path_new = []

for PathPoint in root.iter('PathPoint'):
    point = PathPoint.attrib['name']   
    if point in mscles_df.index:      
        
        location = mscles_df.loc[point, 'location']
        new_text = location[1:-1]
        muscle_path_names.append(point)
        muscle_path_new.append(new_text)
        muscle_path_old.append(PathPoint.find('location').text)

        PathPoint.find('location').text = new_text
    else: pass

In [18]:
muscle_path_new_df = pd.DataFrame([[float(i) for i in point.split()] for point in muscle_path_new], index=muscle_path_names, columns=['x','y','z'])
muscle_path_old_df = pd.DataFrame([[float(i) for i in point.split()] for point in muscle_path_old], index=muscle_path_names, columns=['x','y','z'])

muscle_path_diff_df = muscle_path_old_df - muscle_path_new_df
muscle_path_diff_df['d'] = (muscle_path_diff_df['x']**2+muscle_path_diff_df['y']**2+muscle_path_diff_df['z']**2)**0.5

### Update muscle wrapping surfaces

In [19]:
wrap_cyl_name = []
wrap_transl_old = []
wrap_transl_new = []
wrap_radius_old = []
wrap_radius_new = []

In [20]:
for WrapCylinder in root.iter('WrapCylinder'):
    point = WrapCylinder.attrib['name']
    wrap_cyl_name.append(point)

    if point in wrp_df.index:
        translation = wrp_df.loc[point, 'location']
        wrap_transl_new.append(translation)
        wrap_transl_old.append(WrapCylinder.find('translation').text)
        WrapCylinder.find('translation').text = translation[1:-1]

    else: pass

### Update Translation of joint centres

In [21]:
torso_origin_in_pelvis = markers_df.loc['torso_origin_in_pelvis', 'location']

femur_r_center_in_pelvis = markers_df.loc['femur_r_center_in_pelvis', 'location']
femur_l_center_in_pelvis = markers_df.loc['femur_l_center_in_pelvis', 'location']
knee_l_center_in_femur = markers_df.loc['knee_l_center_in_femur_l', 'location']
knee_r_center_in_femur = markers_df.loc['knee_r_center_in_femur_r', 'location']
patella_l_center_in_femur = markers_df.loc['patella_l_in_femur_l', 'location']
patella_r_center_in_femur = markers_df.loc['patella_r_in_femur_r', 'location']

talus_l_center_in_tibia = markers_df.loc['talus_l_center_in_tibia', 'location']
talus_r_center_in_tibia = markers_df.loc['talus_r_center_in_tibia', 'location']

In [22]:
joint_centers_names = []
joint_centers_new = []
joint_centers_old = []

for joint in root.iter('CustomJoint'):
    
    name = joint.attrib['name']
    if joint.attrib['name'] ==  "back":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = torso_origin_in_pelvis[1:-1]
                joint_centers_names.append("back")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

            else: pass

    if joint.attrib['name'] ==  "hip_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":                
                new_text = femur_r_center_in_pelvis[1:-1]
                joint_centers_names.append("hip_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)           
                frame.find('translation').text = new_text

            else: pass
    
    elif joint.attrib['name'] ==  "hip_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_l_center_in_pelvis[1:-1]
                joint_centers_names.append("hip_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
            else: pass
            
    elif joint.attrib['name'] ==  "walker_knee_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_r_offset":
                new_text = knee_r_center_in_femur[1:-1]
                joint_centers_names.append("walker_knee_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

            #if frame.attrib['name'] == "tibia_r_offset":
            #    new_text = f"{walker_knee_r_tibia_r_offset[0]}, {walker_knee_r_tibia_r_offset[1]}, {walker_knee_r_tibia_r_offset[2]}"
            #    frame.find('translation').text = new_text
            else: pass    
    
    elif joint.attrib['name'] ==  "walker_knee_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_l_offset":
                new_text = knee_l_center_in_femur[1:-1]
                joint_centers_names.append("walker_knee_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text

    #        if frame.attrib['name'] == "tibia_l_offset":
    #            new_text = f"{walker_knee_l_tibia_l_offset[0]}, {walker_knee_l_tibia_l_offset[1]}, {walker_knee_l_tibia_l_offset[2]}"
    #            frame.find('translation').text = new_text
            else: pass

    elif joint.attrib['name'] ==  "patellofemoral_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_r_offset":               
                new_text = knee_r_center_in_femur[1:-1]
                joint_centers_names.append("patellofemoral_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass
    
    elif joint.attrib['name'] ==  "patellofemoral_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_l_offset":              
                new_text = knee_l_center_in_femur[1:-1]
                joint_centers_names.append("patellofemoral_l")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass

for joint in root.iter('PinJoint'):   
    name = joint.attrib['name']
    if joint.attrib['name'] ==  "ankle_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_r_offset":              
                new_text = talus_r_center_in_tibia [1:-1]
                joint_centers_names.append("ankle_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)
                frame.find('translation').text = new_text
           else: pass
    
    elif joint.attrib['name'] ==  "ankle_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_l_offset":              
                new_text = talus_r_center_in_tibia [1:-1]              
                joint_centers_names.append("ankle_r")
                joint_centers_new.append(new_text)
                joint_centers_old.append(frame.find('translation').text)              
                frame.find('translation').text = new_text
           else: pass
    else: pass

### Update Markers for Joint Positions in Parent

In [23]:
m_names = ['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r']

for marker in root.iter('Marker'):
    name = marker.attrib['name']
    if name == 'ankle_l_tibia_l_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['talus_l_center_in_tibia', 'location']
        marker.find('location').text = new_text[1:-1]
    elif name == 'ankle_r_tibia_r_offset':
        old_text = marker.find('location').text
        new_text = markers_df.loc['talus_r_center_in_tibia', 'location']
        marker.find('location').text = new_text[1:-1]
    elif name in m_names:
        old_text = marker.find('location').text
        new_text = markers_df.loc[name, 'location']
        marker.find('location').text = new_text[1:-1]

### Uldate Skin Markers

In [24]:
# symmetrize skin markers
str2lst = lambda x: [y for y in x.strip('[]').split() if y != '']
R_xy = [[1,  0,  0],
        [0,  1,  0],
        [0,  0, -1]]


In [25]:

skin_pelvis = skin_df.loc[skin_df['body'] == 'pelvis', 'location'].apply(str2lst)
skin_pelvis_reflected = skin_pelvis.copy()
skin_pelvis_reflected['RASI'], skin_pelvis_reflected['LASI'] = skin_pelvis['LASI'], skin_pelvis['RASI']
skin_pelvis_reflected['RPSI'], skin_pelvis_reflected['LPSI'] = skin_pelvis['LPSI'],skin_pelvis['RPSI']
skin_pelvis_reflected['PE02'], skin_pelvis_reflected['PE03'] = skin_pelvis['PE03'], skin_pelvis['PE02']

pelvis_symmetric = ((skin_pelvis.apply(lambda x: np.array(x, dtype=float))).to_numpy() + \
(skin_pelvis_reflected.apply(lambda x: (np.array(x, dtype=float)@np.array(R_xy)))).to_numpy())*0.5

skin_pelvis_symmetric = pd.DataFrame(pelvis_symmetric, index=skin_pelvis.index)

In [26]:
skin_femur_r = skin_df.loc[skin_df['body'] == 'femur_r', 'location'].apply(str2lst)
skin_femur_l = skin_df.loc[skin_df['body'] == 'femur_l', 'location'].apply(str2lst)

femur_r_symmetric = ((skin_femur_r.apply(lambda x: np.array(x, dtype=float))).to_numpy() + \
                     (skin_femur_l.apply(lambda x: (np.array(x, dtype=float)@np.array(R_xy)))).to_numpy())*0.5
skin_femur_r_symmetric = pd.DataFrame(femur_r_symmetric, index=skin_femur_r.index)

femur_l_symmetric = ((skin_femur_l.apply(lambda x: np.array(x, dtype=float))).to_numpy() + \
                     (skin_femur_r.apply(lambda x: (np.array(x, dtype=float)@np.array(R_xy)))).to_numpy())*0.5
skin_femur_l_symmetric = pd.DataFrame(femur_l_symmetric, index=skin_femur_l.index)

In [27]:
skin_tibia_r = skin_df.loc[skin_df['body'] == 'tibia_r', 'location'].apply(str2lst)
skin_tibia_l = skin_df.loc[skin_df['body'] == 'tibia_l', 'location'].apply(str2lst)

tibia_r_symmetric = ((skin_tibia_r.apply(lambda x: np.array(x, dtype=float))).to_numpy() + \
                     (skin_tibia_l.apply(lambda x: (np.array(x, dtype=float)@np.array(R_xy)))).to_numpy())*0.5
skin_tibia_r_symmetric = pd.DataFrame(tibia_r_symmetric, index=skin_tibia_r.index)

tibia_l_symmetric = ((skin_tibia_l.apply(lambda x: np.array(x, dtype=float))).to_numpy() + \
                     (skin_tibia_r.apply(lambda x: (np.array(x, dtype=float)@np.array(R_xy)))).to_numpy())*0.5
skin_tibia_l_symmetric = pd.DataFrame(tibia_l_symmetric, index=skin_tibia_l.index)

In [28]:
skin_symm_df = pd.concat([skin_pelvis_symmetric, skin_femur_r_symmetric,skin_femur_l_symmetric,skin_tibia_r_symmetric, skin_tibia_l_symmetric])

In [29]:
for marker in root.iter('Marker'):
    name = marker.attrib['name']
    if name in list(skin_df.index):
        old_text = marker.find('location').text
        new_text = f"{skin_symm_df.loc[name][0][0]} {skin_symm_df.loc[name][0][1]} {skin_symm_df.loc[name][0][2]}"
        marker.find('location').text = new_text
    else: pass

### Export transformed model

In [30]:
tree.write(new_model)

## Scale muscle fibres

In [31]:
# from fibre_scale_script import *

In [32]:
# # define models
# generic_model = '../templates/RajagopalModified_generic_copy.osim'
# model_to_update = f'final_results/tps_transformed.osim'
# updated_model = f'final_results/tps_fibres_updated.osim'

# # run scripts
# osimModel_opt, SimInfo = optimMuscleParams(generic_model, model_to_update, 2, 'final_results/logging')

# # printing the optimized model
# osimModel_opt.printToXML(updated_model)

## Move bone surfaces to the new model folder

In [33]:
bones_folder = os.path.join(path_to_model, 'bones')
os.makedirs(bones_folder, exist_ok=True)

In [34]:
# this code moves the bones from the TPS folder to the bones folder
import shutil
for file in os.listdir(tps_folder):
    if file.endswith('.stl'):
        old_path = os.path.join(tps_folder, file)
        new_path = os.path.join(bones_folder, file)
        print(old_path, new_path)
        shutil.copy(old_path, new_path)

tps_warping_results\results\femur_l.stl 4_tps-bones-muscles-updated\bones\femur_l.stl
tps_warping_results\results\femur_r.stl 4_tps-bones-muscles-updated\bones\femur_r.stl
tps_warping_results\results\fibula_l.stl 4_tps-bones-muscles-updated\bones\fibula_l.stl
tps_warping_results\results\fibula_r.stl 4_tps-bones-muscles-updated\bones\fibula_r.stl
tps_warping_results\results\l_pelvis.stl 4_tps-bones-muscles-updated\bones\l_pelvis.stl
tps_warping_results\results\patella_l.stl 4_tps-bones-muscles-updated\bones\patella_l.stl
tps_warping_results\results\patella_r.stl 4_tps-bones-muscles-updated\bones\patella_r.stl
tps_warping_results\results\r_pelvis.stl 4_tps-bones-muscles-updated\bones\r_pelvis.stl
tps_warping_results\results\sacrum.stl 4_tps-bones-muscles-updated\bones\sacrum.stl
tps_warping_results\results\tibia_l.stl 4_tps-bones-muscles-updated\bones\tibia_l.stl
tps_warping_results\results\tibia_r.stl 4_tps-bones-muscles-updated\bones\tibia_r.stl


## Fit skin markers

In [35]:
# Save skin markers to markers.xml
skin_model = osim.Model(new_model)
skin_model.initSystem()

# Get the marker set
marker_set = skin_model.getMarkerSet()
cloned_marker_set = marker_set.clone()
cloned_marker_set.printToXML(os.path.join(path_to_model, 'markers.xml'))

True

In [36]:
## Fit Skin Markers
# <model_file>
model = new_model # f'final_results/tps_fibres_updated.osim'

# <time_range>'
time_range = f' 0 {list(static_df.loc[static_df.shape[0], "Time"])[0] }'

# <output_scale_file>
path_to_model = '4_tps-bones-muscles-updated'
output_scale_file = os.path.join(path_to_model, f'{ind}_marker_placer.txt')
output_scaling_settings = os.path.join(path_to_model, f'{ind}_marker_placer.xml')
output_model_file = os.path.join(path_to_model, f'{ind}_tps_skin_updated.osim')

# <marker_set_file>
path_to_marker_set = os.path.join(path_to_model, 'markers.xml')

In [37]:
# parse the .xml with markers in body frames
scaling_tree=ET.parse("../templates/marker_placer.xml")
scaling_root = scaling_tree.getroot()

for generic_model in scaling_root.iter('model_file'):
    generic_model.text = model
for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range
for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file
for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

In [38]:
scaling_tree.write(output_scaling_settings)

In [ ]:
from simFunctions import runProgram

cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = output_scaling_settings
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

## Update Wrapping Surfaces

In [ ]:
from wrap_scripts import *

model_to_update = output_model_file
updated_model = os.path.join(path_to_model, f'{ind}_tps_skin_wrp_updated.osim')

# function to extract muscle paths, wrapping surfaces info and joint info from an osim model
# returns three dataframes: muscles, surfaces and joints
fix_wraps = FixWraps(model_to_update)

In [41]:
wrp_adjust_dict = fix_wraps.collect_wrp_details()
wrp_adjust_df = pd.DataFrame.from_dict(wrp_adjust_dict)
wrp_adjust_df.set_index(pd.Index(['body', 'transl', 'rad', 'point_label', 'point', 'point_body', 'joint_location', 'joint_location+point', 'dist_to_transl',  'dist']), inplace = True)


In [ ]:
wrp_adjust_df

In [ ]:
#print points whose location is too far into radius for an automatic adjustment
manual = []
for col in wrp_adjust_df.columns:
    if wrp_adjust_df.loc['dist', col] < wrp_adjust_df.loc['rad', col]-0.005:
        print(col, ':', wrp_adjust_df.loc['point_label', col])
        manual.append(col)

# ignore patella points for now
# adjust other points or wrapping surfaces manually for the input model and run the script again

In [ ]:
# # update model
tree_model = ET.parse(model_to_update)
root = tree_model.getroot()

wrp_to_change = [i for i in wrp_adjust_df.columns if i not in manual]

for WrapCylinder in root.iter('WrapCylinder'):
    name = WrapCylinder.attrib['name']
    if name in wrp_to_change:
        print(name)
        radius = wrp_adjust_df.loc['rad', name]
        print('old_radius', radius)
        new_rad = wrp_adjust_df.loc['dist', name]
        print('new_radius', new_rad)
        radius_new_text = str(new_rad-0.01)
        WrapCylinder.find('radius').text = radius_new_text

tree_model.write(updated_model)